## Description

The artist Piet Mondrian is a famous mid-century abstract artist. His designs of brightly colored rectangles on a canvas should be familiar to you even if you don't know his name. He's even given his name to a visual programming language Piet.

I learned about this puzzle from this video from TED-Ed on the challenge. Briefly:

"Fit non-congruent rectangles into a n*n square grid. What is the smallest difference possible between the areas of the largest and the smallest rectangles?"

Remember a non-congruent rectangle is a shape with distinct measurements, so a 8x1 rectangle is the same as a 1x8, but distinct from a 2x4.

Your challenge today is to write a program that can heuristically subdivide the canvas and find a minimal area range.

This is sequence A276523 in the OEIS database.

## Input Description

You'll be given an integer n, one per line. This is the size of your canvas to work with. Example:

    11

## Output Description

Your program should emit the smallest value you can find for that canvas size, optionally the dimensions of the rectangles your program generated. Example:
    
    6
    3 X 4
    2 X 6
    2 X 7
    3 X 5
    4 X 4
    2 X 8
    2 X 9
    3 X 6

## Challenge Input

    4
    8
    10
    20
    25
    32

## Bonus Input

Note that solutions above n=44 don't yet have a known or proven lower bound.

    50

## First iteration

Retrieve existing solutions

In [1]:
import requests

def call_oeis(id_):
    r = requests.get(f'http://oeis.org/A{id_}/b{id_}.txt')
    inx = [x.split() for x in r.content.decode('UTF-8').splitlines()]
    s = [[int(x) for x in y] for y in inx if len(y) == 2]
    D = {x:y for x,y in s}
    return D

class Mondriaan(object):
    def __init__(self):
        self.known = call_oeis(276523)

    def search(self,x):
        if x in self.known:
            return self.known[x]
        return search_unknown(self,x)

    def search_unknown(self,x):
        return '?'

M = Mondriaan()
print(' '.join([str(M.search(x)) for x in [4,8,10,11,20,25,32,50]]))

4 6 8 6 9 10 10 13


## Second Iteration

Retrieve existing solutions. Use these to generate an actual result.

In [2]:
from itertools import combinations_with_replacement as cwr
from rectpack import packer

M = Mondriaan()

def rectangles(n):
    return [(x*y,(x,y)) for x,y in cwr(range(1,n+1),2)][:-1]

def surface_ranges(n):   
    r = sorted(rectangles(n),key = lambda x:x[0])
    s = M.search(n)
    p = []
    for ix,(vx,rx) in enumerate(r):
        for iy,(vy,ry) in enumerate(r):
            v = r[ix:iy+1]
            if vy - vx == s \
            and sum(x[0] for x in v) >= n**2 \
            and sum(x[0] for x in v[:2]) <= n**2:
                p.append(v)
            if vy - vx > s:
                break
    return p

def pack_them(a_p,n):
    ordered = packer.SORT_AREA([y for x,y in a_p])

    P = packer.PackerGlobal()
    P.add_bin(n,n,count=float('inf'))
    for x in ordered:
            P.add_rect(*x)
    P.pack()
    a = [x for x in P.rect_list() if x[0] == 0]
    if sum(l*h for b,x,y,l,h,n in a) == n*n:
            return a

def run(n):
    p = surface_ranges(n)
    for x in p:
        s = pack_them(x,n)
        if s:
            return [(l,h) for b,x,y,l,h,desc in s]

for x in range(3,16):
    print(x,run(x))

3 [(1, 3), (2, 2), (2, 1)]
4 [(1, 4), (3, 2), (2, 2), (1, 2)]
5 [(2, 5), (3, 3), (3, 2)]
6 [(1, 6), (5, 1), (2, 4), (3, 2), (4, 1), (2, 2), (1, 3)]
7 [(1, 7), (6, 1), (1, 5), (4, 1), (2, 4), (3, 3), (3, 2), (2, 2)]
8 [(1, 8), (7, 2), (2, 6), (5, 2), (3, 4), (2, 4)]
9 [(2, 9), (7, 3), (4, 6), (3, 6)]
10 None
11 [(2, 9), (8, 2), (3, 6), (3, 5), (6, 2), (2, 7), (4, 4), (4, 3)]
12 [(1, 12), (11, 1), (2, 9), (8, 2), (3, 6), (3, 5), (6, 2), (2, 7), (4, 4), (4, 3)]
13 None
14 None
15 [(3, 15), (12, 4), (4, 11), (8, 6), (8, 5)]


## Third Iteration

If all else fails, try a little randomness (Given a correct list, one or more off the sorts will lend itself to easy packing, shuffling beats enumerating the permutations). Tends to find solutions but not always. Could add some more retries, but seems futile: the costs increase so fast.

Changes:

- pack only rectangles for which total surface == square
- use set of tuples to avoid doubles
- added up to 200 random sort attempt per possible collection of rectangles

In [3]:
from itertools import combinations, chain, combinations_with_replacement as cwr
from rectpack import packer
from time import time
import random

t = time()
M = Mondriaan()

def rectangles(n):
    return [(x*y,(x,y)) for x,y in cwr(range(1,n+1),2)][:-1]

def surface_ranges(n):   
    r = sorted(rectangles(n),key = lambda x:x[0])
    s = M.search(n)
    p = []
    for ix,(vx,rx) in enumerate(r):
        for iy,(vy,ry) in enumerate(r):
            v = r[ix:iy+1]
            if vy - vx == s \
            and sum(x[0] for x in v) >= n**2 \
            and sum(x[0] for x in v[:2]) <= n**2:
                p.append(v)
            if vy - vx > s:
                break
    return p

def give_change(change, coins):
    subsets = [combinations(coins, x) for x in range(len(coins))]
    return [[y[1] for y in x] for x in chain(*subsets) if sum(y[0] for y in x) == change]

def perfect_ranges(p,n):
    # use Set to avoid doubles after give_change cleanup
    ranges = set()
    s = n**2
    for x in p:
        v = sum(y for y,z in x)
        diff = v-s
        if diff:
            changes = give_change(diff,x)
            if changes:
                for change in changes:
                    p = tuple(b for a,b in x if b not in change)
                    ranges.add(p)
        else:
            ranges.add(tuple(b for a,b in x))   
    return list(ranges)

def backup(S,n,retry=True):
    for x in S:
        c = 100
        b = list(x)
        while c:
            random.shuffle(b)
            P = packer.newPacker(mode=packer.PackingMode.Online)
            P.add_bin(n,n,count=float('inf'))
            for x in b:
                P.add_rect(*x)
            a = [x for x in P.rect_list() if x[0] == 0]
            if sum(l*h for b,x,y,l,h,n in a) == n*n:
                return b
            c -= 1
    if retry: 
        return backup(S,n,False)

def pack_them(a_p,n,sort_):
    ordered = sort_(a_p)
    P = packer.PackerGlobal()
    P.add_bin(n,n,count=float('inf'))
    for x in ordered:
            P.add_rect(*x)
    P.pack()
    a = [x for x in P.rect_list() if x[0] == 0]
    if sum(l*h for b,x,y,l,h,n in a) == n*n:
            return a
    
def run(n):
    p = surface_ranges(n)
    pr = perfect_ranges(p,n)
    for x in pr:
        s = pack_them(x,n,packer.SORT_AREA)
        if s:
            return [(l,h) for b,x,y,l,h,desc in s]
    return backup(pr,n)

for x in range(3,18):
    print(x,run(x))
    
print(f"\n{time()-t:.3f} s")

3 [(1, 3), (2, 2), (2, 1)]
4 [(1, 4), (3, 2), (2, 2), (1, 2)]
5 [(2, 5), (3, 3), (3, 2)]
6 [(1, 5), (4, 1), (2, 4), (3, 3), (3, 2), (2, 2)]
7 [(1, 7), (6, 1), (1, 5), (4, 1), (2, 4), (3, 3), (3, 2), (2, 2)]
8 [(1, 8), (7, 2), (2, 6), (5, 2), (3, 4), (2, 4)]
9 [(3, 9), (6, 5), (6, 4)]
10 [(2, 10), (2, 6), (2, 7), (2, 8), (4, 5), (3, 6)]
11 [(2, 9), (8, 2), (3, 6), (3, 5), (6, 2), (2, 7), (4, 4), (4, 3)]
12 [(1, 12), (11, 1), (2, 9), (8, 2), (3, 6), (3, 5), (6, 2), (2, 7), (4, 4), (4, 3)]
13 [(2, 13), (4, 8), (3, 10), (3, 9), (3, 8), (5, 6)]
14 [(3, 10), (3, 11), (4, 8), (5, 7), (6, 6), (5, 6)]
15 [(3, 15), (12, 4), (4, 11), (8, 6), (8, 5)]
16 [(2, 16), (2, 14), (3, 10), (3, 11), (5, 7), (6, 6), (5, 6), (4, 8)]
17 None

46.027 s
